## About this notebook

* No ensembling is used
* Single efficientnet-b2 model is trained(achieved **0.921 score on public LB**)
* Anyone with a kaggle account can reproduce the results
* Trained only on the gpu provided by the kaggle platform
* No cross validation used
* Minimum data augmentation is applied

The inference notebook can be found [here](https://www.kaggle.com/bipinkrishnan/ranzcr-clip-inference-notebook)

In [ ]:
from PIL import Image
import sys

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torch
from torch import optim
from torchvision.transforms import transforms
from torch import nn
from torch.utils.data import DataLoader, Dataset

sys.path.append('../input/efficient-net-deps-1/')
from efficientnet_pytorch.model import EfficientNet
from tqdm.notebook import tqdm

In [ ]:
root_path = '../input/ranzcr-clip-catheter-line-classification/'
train_imgs = root_path + 'train/'
test_imgs = root_path+ 'test/'

df = pd.read_csv(root_path+'train.csv')
submission = pd.read_csv(root_path+'sample_submission.csv')

In [ ]:
train_csv, val_csv = train_test_split(df, test_size=0.2, random_state=42)
len(train_csv), len(val_csv)

In [ ]:
class LoadData(Dataset):
    def __init__(self, df, transform, test=False):
        super().__init__()
        self.df = df
        self.transform = transform
        self.test = test
        
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        if not self.test:
            img = Image.open(train_imgs+row[0]+'.jpg').convert('RGB')
            labels = torch.from_numpy(row[1:-1].astype(np.float32).values)
        
            return self.transform(img), labels 
        else:
            img = Image.open(test_imgs+row[0]+'.jpg').convert('RGB')
            return self.transform(img)
        
    def __len__(self): return len(self.df)

In [ ]:
class Trainer(nn.Module):
    def __init__(self, device, model_name, scheduler):
        super().__init__()
        self.model_name = model_name
        self.device = device
        self.model = self.build_model().to(self.device)
        self.criterion = nn.BCEWithLogitsLoss()
        self.scheduler = scheduler
        
        if scheduler=='reducelronplat':
            self.opt = optim.Adam(self.model.parameters())
            self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.opt, 
                                                                  mode='max', 
                                                                  patience=1, 
                                                                  verbose=True)
        elif scheduler=='cosineannealing':
            self.opt = optim.Adam(self.model.parameters(), lr=1e-4)
            self.scheduler = optim.lr_scheduler.CosineAnnealingLR(self.opt, 
                                                                  T_max=3,
                                                                  verbose=False)
            
        
    def build_model(self):
        #building the model for transfer learning
        if self.model_name=='effnet-b5':
            effnet = EfficientNet.from_pretrained('efficientnet-b5', num_classes=11)
            return effnet
        elif self.model_name=='effnet-b2':
            effnet = EfficientNet.from_pretrained('efficientnet-b2', num_classes=11)
            return effnet
        
    def train_loop(self, data, label):
        data, label = data.to(self.device), label.to(self.device)
        self.opt.zero_grad()
        train_out = self.model(data)
        train_loss = self.criterion(train_out, label)
        train_loss.backward()
        self.opt.step()
        
        return train_loss, train_out
    
    def val_loop(self, val_data, val_label):
        val_data, val_label = val_data.to(self.device), val_label.to(self.device)
        val_out = self.model(val_data)
        val_loss = self.criterion(val_out, val_label)
        
        return val_loss, val_out
            
    def freeze_fit(self, epoch, train_dl):
        print("-------------Starting freezed fit-------------")
        if self.model_name=='effnet-b5':
            self.model = self.unfreeze_linear(self.freeze_all(self.model), 2048, 11)
        elif self.model_name=='effnet-b2':
            self.model = self.unfreeze_linear(self.freeze_all(self.model), 1408, 11)
        self.model.train()
        for i in range(epoch):
            for data, label in tqdm(train_dl, total=len(train_dl), leave=False):
                loss, out = self.train_loop(data, label)
            print(f"Epoch: {i+1}/{epoch}  train_loss: {loss}")            
        
    def fit(self, epochs, train_dl, val_dl):
        #unfreeze the model and
        #training for specified number of epochs
        print("-------------Starting unfreezed fit-------------")
        self.model = self.unfreeze_all(self.model)
        current_score = 0.0
        for epoch in range(epochs):
            val_preds, val_labels = [], []
            self.model.train()
            for i, (data, label) in enumerate(tqdm(train_dl, total=len(train_dl), leave=False), 1):
                train_loss, train_out = self.train_loop(data, label)
            
            self.model.eval()
            with torch.no_grad():
                for j, (val_data, val_label) in enumerate(tqdm(val_dl, total=len(val_dl), leave=False), 1):
                    val_loss, val_out = self.val_loop(val_data, val_label)
                    val_preds.append(val_out.cpu())
                    val_labels.append(val_label.cpu())
            
            val_preds, val_labels = np.concatenate(val_preds), np.concatenate(val_labels)
            avg_score, scores = self.get_score(val_preds, val_labels)
            if self.scheduler=='reducelronplat':
                self.scheduler.step(avg_score)
            elif self.scheduler=='cosineannealing':
                self.scheduler.step()
            
            if avg_score > current_score:
                torch.save(self.model.state_dict(), "model.pt")
                current_score = avg_score

            print(f"{epoch+1}/{epochs}  train_loss: {train_loss}  val_loss: {val_loss}  score: {avg_score}")
    
    def get_score(self, preds, labels):
        #Calculates ROC AUC score
        scores = []
        for i in range(labels.shape[1]):
            score = roc_auc_score(labels[:, i], preds[:, i])
            scores.append(score)
        avg_score = np.mean(scores)
        return avg_score, scores
            
    def load_model(self, path):
        return self.model.load_state_dict(torch.load(path))
          
    def freeze_all(self, model):
        #freezes all layers of the model
        for params in model.parameters():
            params.requires_grad = False
        return model
    
    def unfreeze_linear(self, model, in_features, out_features):
        #unfreezes the specified layers of the model
        for params in model.parameters():
            if params.shape==torch.Size([out_features, in_features]) or params.shape==torch.Size([out_features]):
                params.requires_grad = True
        return model
    
    def unfreeze_all(self, model):
        #unfreezes all layers of the model
        for params in model.parameters():
            params.requires_grad = True
        return model        

In [ ]:
BATCH_SIZE = 32
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
IMG_SIZE = 256

transform = transforms.Compose([transforms.Resize((IMG_SIZE, IMG_SIZE)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])

train_ds = LoadData(train_csv, transform, test=False)
val_ds = LoadData(val_csv, transform, test=False)
test_ds = LoadData(submission, transform, test=True)

train_dl = DataLoader(train_ds, BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, BATCH_SIZE, shuffle=False)
test_dl = DataLoader(test_ds, BATCH_SIZE, shuffle=False)

## Training the model

In [ ]:
trainer = Trainer(device=DEVICE, model_name='effnet-b2', scheduler='reducelronplat')

In [ ]:
trainer.freeze_fit(1, train_dl)
trainer.fit(5, train_dl, val_dl)

In [ ]:
model_path = './model.pt'
trainer.opt.param_groups[0]['lr'] = 1e-4
trainer.load_model(model_path)

In [ ]:
trainer.fit(5, train_dl, val_dl)